<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Harvest - Get Filtered List of Time Entries
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Harvest/Harvest_List_all_clients.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedbacks</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #harvest #timeentries #api #list #python #get #filter

**Author:** [Landry Christensen](https://github.com/lchristensen6)

**Last update:** 2023-08-01 (Created: 2023-08-01)

**Description:** This notebook will create a filtered list of time entries from the Harvest API v2. It is usefull for organizations to quickly access and display time entries based on a specific filter, such as by time period or project.

**References:**
- [Harvest API v2 - Time Entries](https://help.getharvest.com/api-v2/timesheets-api/timesheets/time-entries/)
- [Harvest API v2 - Authentication](https://help.getharvest.com/api-v2/authentication-api/authentication/authentication/)

## Input

### Import libraries

In [1]:
import requests
import pandas as pd
import naas

### Setup Variables
[Create your personnal access tokens](https://id.getharvest.com/oauth2/access_tokens/new)
- `account_id`: Account ID from Harvest
- `access_token`: Access token from Harvest
- `limit`: Entries limit, to get all entries enter -1

In [2]:
account_id = naas.secret.get("HARVEST_ACCOUNT_ID") or "YOUR_HARVEST_ACCOUNT_ID"
access_token = naas.secret.get("HARVEST_ACCESS_TOKEN") or "YOUR_HARVEST_ACCESS_TOKEN"
limit = 1000

### Create filters
- `user_id`: User ID from Harvest
- `client_id`: Client ID from Harvest
- `project_id`: Project ID from Harvest
- `task_id`: Task ID from Harvest
- `date_from`: Starting date to filter from ([ISO 8601 Format](https://en.wikipedia.org/wiki/ISO_8601))
- `date_to`: Ending date to filter from ([ISO 8601 Format](https://en.wikipedia.org/wiki/ISO_8601))

Leave a variable as a blank string to remove a specific filter

In [70]:
user_id = 'USER_ID'
client_id = 'CLIENT_ID'
project_id = 'PROJECT_ID'
task_id = 'TASK_ID'
date_from = 'DATE_FROM' # Example date '2023-08-01'
date_to = 'DATE_TO' # Example date '2023-08-01'

## Model

### List all time entries

This function will list filtered time entries from the Harvest API v2.

In [71]:
# Flatten the nested dict
def flatten_dict(d, parent_key='', sep='_'):
    """
    Flattens a nested dictionary into a single level dictionary.

    Args:
        d (dict): A nested dictionary.
        parent_key (str): Optional string to prefix the keys with.
        sep (str): Optional separator to use between parent_key and child_key.

    Returns:
        dict: A flattened dictionary.
    """
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def list_time_entries(account_id, access_token, limit=-1, user_id='', client_id='', 
                      project_id='', task_id='', date_from='', date_to=''):
    # Init
    data = []
    df = pd.DataFrame()
    
    
    # Requests
    url = f"https://api.harvestapp.com/v2/time_entries?account_id={account_id}"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Harvest-Account-Id": account_id,
        "User-Agent": "Harvest API Python Client",
        "Content-Type": "application/json",
    }
    params = {
        "user_id": user_id,
        "client_id": client_id,
        "project_id": project_id,
        "task_id": task_id,
        "from": date_from,
        "to": date_to,
    }
    
    # Loop on while
    while True:
        res = requests.get(url, headers=headers, params=params)
        if res.status_code == 200:
            # Get data
            res_json = res.json()
            time_entries = res_json.get("time_entries")
            for time_entry in time_entries:
                data.append(flatten_dict(time_entry))
            
                # Manage limit
                if limit != -1 and len(data) >= limit:
                    break
                
            # Check next link
            link_next = res_json.get("links").get("next")
            if link_next:
                url = link_next
            else:
                break
            
    # Transform in dataframes
    df = pd.DataFrame(data)
    return df

## Output

### Display result

In [73]:
df_time_entries = list_time_entries(
    account_id,
    access_token,
    limit,
    user_id = user_id,
    client_id = client_id,
    project_id = project_id,
    task_id = task_id,
    date_from = date_from,
    date_to = date_to
)

print("Row fetched:", len(df_time_entries))
df_time_entries.head()

Row fetched: 481


,id,spent_date,hours,hours_without_timer,rounded_hours,notes,is_locked,locked_reason,is_closed,is_billed,timer_started_at,started_time,ended_time,is_running,billable,budgeted,billable_rate,cost_rate,created_at,updated_at,user_id,user_name,client_id,client_name,client_currency,project_id,project_name,project_code,task_id,task_name,user_assignment_id,user_assignment_is_project_manager,user_assignment_is_active,user_assignment_use_default_rates,user_assignment_budget,user_assignment_created_at,user_assignment_updated_at,user_assignment_hourly_rate,task_assignment_id,task_assignment_billable,task_assignment_is_active,task_assignment_created_at,task_assignment_updated_at,task_assignment_hourly_rate,task_assignment_budget,invoice,external_reference
0,2143681821,2023-08-01,3.50,3.50,3.50,draft Purchase agreements for additional 250k ...,False,None,False,False,None,None,None,False,True,False,NaN,None,2023-08-01T23:37:13Z,2023-08-01T23:37:13Z,4572674,Darren Watts,13551915,OpenTeams Incubator,USD,36285860,Legal,OSBIG-COST,20342578,OpenSource Capital LLC,397857645,False,True,True,None,2023-04-03T16:18:18Z,2023-04-03T16:18:18Z,NaN,390761422,True,True,2023-03-16T20:10:09Z,2023-03-16T20:10:09Z,NaN,None,None,None
1,2143665938,2023-08-01,11.00,11.00,11.00,None,False,None,False,False,None,None,None,False,True,False,NaN,None,2023-08-01T23:06:25Z,2023-08-01T23:16:10Z,4572684,Inessa Pawson,13551916,"OpenTeams, Inc.",USD,36798286,Project Success Management,OPENTEAMS-INTER,20607878,Quansight Consulting,403325742,False,True,True,None,2023-05-02T15:47:33Z,2023-05-02T15:47:33Z,NaN,396645702,True,True,2023-05-02T15:47:33Z,2023-05-02T15:47:33Z,NaN,None,None,None
2,2143665935,2023-08-01,0.75,0.75,0.75,None,False,None,False,False,None,None,None,False,True,False,85.0,None,2023-08-01T23:06:25Z,2023-08-01T23:06:39Z,4572684,Inessa Pawson,13623598,NumFocus,USD,36495395,CZI / DEI Grant,OSBIG-RATE,20099592,Programming,398130516,False,True,False,None,2023-04-04T19:23:37Z,2023-04-04T19:23:37Z,85.0,393228630,True,True,2023-04-04T19:23:37Z,2023-04-04T19:23:37Z,NaN,None,None,None
3,2143655895,2023-08-01,1.09,0.00,1.09,,False,None,False,False,2023-08-01T22:50:15Z,None,None,True,True,False,0.0,None,2023-08-01T22:50:15Z,2023-08-01T23:48:41Z,4660599,Landry Christensen,13551915,OpenTeams Incubator,USD,36285867,INTER RPA Support,OSBIG-INTER,20342621,OpenTeams,416681835,False,True,True,None,2023-07-27T22:53:02Z,2023-07-27T22:53:02Z,NaN,390762781,True,True,2023-03-16T20:19:46Z,2023-03-16T20:19:46Z,0.0,None,None,None
4,2143651465,2023-08-01,0.00,0.00,0.00,,False,None,False,False,None,None,None,False,True,True,125.0,None,2023-08-01T22:42:57Z,2023-08-01T22:42:57Z,4564132,Melanie Maxfield,13710834,"NaasAI, Inc.",USD,36781551,RATE Accounting,OSBIG-RATE,20342617,Finance / Accounting,403057791,False,True,False,None,2023-05-01T16:42:42Z,2023-05-01T16:42:42Z,125.0,396450600,True,True,2023-05-01T17:24:13Z,2023-05-01T17:24:13Z,NaN,None,None,None
5,2143651464,2023-08-01,0.38,0.38,0.38,,False,None,False,False,None,None,None,False,False,False,NaN,None,2023-08-01T22:42:57Z,2023-08-01T23:05:40Z,4564132,Melanie Maxfield,13551915,OpenTeams Incubator,USD,36285863,INTER Accounting,OSBIG-INTER,20342630,Quansight Labs,397861200,False,True,True,None,2023-04-03T16:28:56Z,2023-04-03T16:28:56Z,NaN,390762043,False,True,2023-03-16T20:13:33Z,2023-04-03T16:29:54Z,NaN,None,None,None
6,2143651462,2023-08-01,0.00,0.00,0.00,,False,None,False,False,None,None,None,False,False,False,NaN,None,2023-08-01T22:42:57Z,2023-08-01T22:42:57Z,4564132,Melanie Maxfield,13551915,OpenTeams Incubator,USD,36285863,INTER Accounting,OSBIG-INTER,20342629,Quansight,397861200,False,True,True,None,2023-04-03T16:28:56Z,2023-04-03T16:28:56Z,NaN,390762041,False,True,2023-03-16T20:13:33Z,2023-04-03T16:29:54Z,NaN,None,None,None
7,2143651461,2023-08-01,0.00,0.00,0.00,,False,None,False,False,None,None,None,False,False,False,NaN,None,2023-08-01T22:42:57Z,2023-08-01T22:42:57Z,4564132,Melanie Maxfield,13551915,OpenTeams Incubator,USD,36285863